In [2]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \n    \'// setup cpp code highlighting\\n\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-cmake"] = {\\\'reg\\\':[/^%%cmake/]} ;\'\n)\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE, STDOUT, check_output\nimport html\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\nimport time\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    parser.add_argument("--under-spoiler-threshold", type=int, default=None)\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                comment_prefix = "%" + line_comment_start\n                if line.startswith(comment_prefix):\n                    cmds.append(\'#\' + line[len(comment_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        if cmd.startswith(\'#\'):\n            display(Markdown("\\#\\#\\#\\# `%s`" % cmd[1:]))\n        else:\n            display(Markdown("Run: `%s`" % cmd))\n            if args.under_spoiler_threshold:\n                out = check_output(cmd, stderr=STDOUT, shell=True, universal_newlines=True)\n                out = out[:-1] if out.endswith(\'\\n\') else out\n                out = html.escape(out)\n                if len(out.split(\'\\n\')) > args.under_spoiler_threshold:\n                    out = "<details> <summary> output </summary> <pre><code>%s</code></pre></details>" % out\n                elif out:\n                    out = "<pre><code>%s</code></pre>" % out\n                if out:\n                    display(HTML(out))\n            else:\n                get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef cmake(fname, cell):\n    save_file(fname, cell, "#")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    fname = fname or "makefile"\n    assert fname.endswith("makefile")\n    save_file(fname, cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    line = line.strip() \n    if line[0] == \'#\':\n        display(Markdown(line[1:].strip()))\n    else:\n        try:\n            expr, comment = line.split(" #")\n            display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n        except:\n            display(Markdown("{} = {}".format(line, eval(line))))\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def wait_stop(self, timeout):\n        for i in range(int(timeout * 10)):\n            wpid, status = os.waitpid(self.pid, os.WNOHANG)\n            if wpid != 0:\n                return True\n            time.sleep(0.1)\n        return False\n        \n    def close(self, timeout=3):\n        self.inq_f.close()\n        if not self.wait_stop(timeout):\n            os.kill(self.get_pid(), signal.SIGKILL)\n            os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Низкоуровневый ввод-вывод

<p><a href="https://www.youtube.com/watch?v=c3rUDPA9Ocs&list=PLjzMm8llUm4AmU6i_hPU0NobgA4VsBowc&index=11" target="_blank">
    <h3>Видеозапись семинара</h3>
</a></p>


[Ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/file_io) 


Сегодня в программе:
* <a href="#linux" style="color:#856024"> Linux </a>  
  * <a href="#read" style="color:#856024"> Чтение из stdin и файлов </a>  
  * <a href="#write" style="color:#856024"> Запись в stdout/stderr и файл </a>
  * <a href="#attrs" style="color:#856024"> Атрибуты файлов </a>  
  * <a href="#lseek" style="color:#856024"> Произвольный доступ к файлам (lseek) </a>
  * <a href="#lsof" style="color:#856024"> Список открытых файлов (lsof) </a>
* <a href="#win" style="color:#856024"> Windows </a>
* <a href="#hw" style="color:#856024"> Комментарии к ДЗ </a>



## <a name="linux"></a> Linux

Здесь полезно рассматривать процесс как объект в операционной системе. Помимо основного пользовательского потока выполнения у процесса-объекта есть множество атрибутов.

Советую прочитать [статью на хабре](https://habr.com/ru/post/423049/#definition), вроде там все очень неплохо написано.

Сегодня нас будут интересовать файловые дескрипторы. Каждому открытому файлу и соединению соответствует число (int). Это число используется как идентификатор в функциях, работающих с файлами/соединениями.


* 0 - stdin - стандартный поток ввода (STDIN_FILENO - стандартный макрос в C)
* 1 - stdout - стандартный поток вывода (STDOUT_FILENO)
* 2 - stderr - стандартный поток ошибок (STDERR_FILENO)

Примеры использования в bash:

* `grep String < file.txt` <-> `grep String 0< file.txt`
* `mkdir a_dir 2> /dev/null`
* `./some_program < in.txt 1> out.txt` <-> `./some_program < in.txt > out.txt` 



## <a name="read"></a> Read 

In [3]:
%%cpp linux_example_read.c
%run gcc linux_example_read.c -o linux_example_read.exe
%run echo -n "Hello from file!" > linux_example_read.txt
%run echo -n "Hello from stdin!" | ./linux_example_read.exe linux_example_read.txt

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>

int main(int argc, char *argv[])
{
    assert(argc >= 2);
    const char* file_name = argv[1];
    
    char buffer[4096];
    int bytes_read = read(STDIN_FILENO, buffer, sizeof(buffer));
    printf("From stdin: '%.*s'\n", bytes_read, buffer); // buffer not zero-terminated string after `read`!
    
    int fd = open(file_name, O_RDONLY); // O_RDWR also works 
    bytes_read = read(fd, buffer, sizeof(buffer));
    printf("From file '%s': '%.*s'\n", file_name, bytes_read, buffer); 
    close(fd);
    
    return 0;
}

Run: `gcc linux_example_read.c -o linux_example_read.exe`

Run: `echo -n "Hello from file!" > linux_example_read.txt`

Run: `echo -n "Hello from stdin!" | ./linux_example_read.exe linux_example_read.txt`

From stdin: 'Hello from stdin!'
From file 'linux_example_read.txt': 'Hello from file!'


**Но в данном процессе чтения есть проблема, не все данные могут быть доступны для чтения сразу.**

In [91]:
!(echo -n "A" ; sleep 1 ; echo -n "B") ; echo "" 

!(echo -n "A" ; sleep 1 ; echo -n "B" 2>/dev/null) | ./linux_example_read.exe linux_example_read.txt

AB
From stdin: 'A'
From file 'linux_example_read.txt': 'Hello from file!'


Поэтому надо делать ретраи.

In [92]:
%%cpp retry_example.c
%run gcc -D_USE_READ retry_example.c -o retry_example.exe
%run echo -n "Hello_world_1!" | ./retry_example.exe 
%run gcc -D_USE_READ retry_example.c -o retry_example.exe
%run (echo -n "Hello_" ; sleep 0.2 ; echo -n "world_2!") | ./retry_example.exe  
%run gcc -D_USE_READ_RETRY retry_example.c -o retry_example.exe
%run (echo -n "Hello_" ; sleep 0.2 ; echo -n "world_3!") | ./retry_example.exe  
%run gcc -D_USE_SCANF retry_example.c -o retry_example.exe
%run (echo -n "Hello_" ; sleep 0.2 ; echo -n "world_4!") | ./retry_example.exe  

#include <unistd.h>
#include <stdio.h>
#include <errno.h>
#include <string.h>


int read_retry(int fd, char* data, int size) {
    char* cdata = data;
    while (1) {
        int read_bytes = read(fd, cdata, size);
        if (read_bytes == 0) { // если read вернул 0, значит файловый дескриптор закрыт с другого конца 
                               // или конец файла
            return cdata - data;
        }
        if (read_bytes < 0) { // если возвращено значение < 0, то это ошибка
            if (errno == EAGAIN || errno == EINTR) { // она может быть retryable
                continue;
            } else { // а может быть критичной, и нет смысла пытаться повторить попытку чтения
                return -1;
            }
        }
        // если возвращенное значение > 0, значит успешно прочитано столько байт
        cdata += read_bytes;
        size -= read_bytes;
        if (size == 0) {
            return cdata - data;
        }
    }
}


int main(int argc, char *argv[])
{ 
    char buffer[4096];
#if defined(_USE_READ) || defined(_USE_READ_RETRY)
  #if defined(_USE_READ)
    int bytes_read = read(0, buffer, sizeof(buffer)); 
  #endif
  #if defined(_USE_READ_RETRY)
    int bytes_read = read_retry(0, buffer, sizeof(buffer)); 
  #endif    
    if (bytes_read < 0) {
        perror("Error reading file");
        return -1;
    }
#endif
#if defined(_USE_SCANF)
    scanf("%s", buffer);
    int bytes_read = strlen(buffer);
#endif  
    printf("Read '%.*s'", bytes_read, buffer);
    
    return 0;
}

Run: `gcc -D_USE_READ retry_example.c -o retry_example.exe`

Run: `echo -n "Hello_world_1!" | ./retry_example.exe`

Read 'Hello_world_1!'

Run: `gcc -D_USE_READ retry_example.c -o retry_example.exe`

Run: `(echo -n "Hello_" ; sleep 0.2 ; echo -n "world_2!") | ./retry_example.exe`

...

Run: `(echo -n "Hello_" ; sleep 0.2 ; echo -n "world_3!") | ./retry_example.exe`

Read 'Hello_world_3!'

Run: `gcc -D_USE_SCANF retry_example.c -o retry_example.exe`

Run: `(echo -n "Hello_" ; sleep 0.2 ; echo -n "world_4!") | ./retry_example.exe`

Read 'Hello_world_4!'

Как можно заметить, scanf молодец. А read надо ретраить.

## <a name="write"></a> Write

С write есть та же проблема, что и с read. Ретраится примерно точно так же.

In [93]:
%%cpp linux_example.c
%run gcc linux_example.c -o linux_example.exe
%run ./linux_example.exe linux_example_input_002.txt 
%run cat linux_example_input_002.txt 

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>
#include <string.h>

int main(int argc, char *argv[])
{
    const char to_stdout[] = "Hello to stdout!\n";
    const char to_stderr[] = "Hello to stderr!\n";
    const char to_file[] = "Hello to file!\n";
    
    assert(argc >= 2);
    const char* file_name = argv[1];
    
    int bytes_written = write(STDOUT_FILENO, to_stdout, strlen(to_stdout));
    assert(bytes_written == strlen(to_stdout));
    
    bytes_written = write(STDERR_FILENO, to_stderr, strlen(to_stderr));
    assert(bytes_written == strlen(to_stderr));
    
    int fd = open(file_name, O_WRONLY | O_CREAT, 0644); // 0644 это важно! Но об этом позже 
    // open(file_name, O_WRONLY | O_CREAT) ~ printf("%s")
    bytes_written = write(fd, to_file, strlen(to_file));
    close(fd);
    
    return 0;
}

Run: `gcc linux_example.c -o linux_example.exe`

Run: `./linux_example.exe linux_example_input_002.txt`

Hello to stdout!
Hello to stderr!


Run: `cat linux_example_input_002.txt`

Hello to file!


### Общий пример

In [94]:
%%cpp linux_example.c
%run gcc linux_example.c -o linux_example.exe
%run echo -n "Hello students!" > linux_example_input_001.txt
%run ./linux_example.exe linux_example_input_001.txt

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[])
{
    printf("Linux by printf"); // where it will be printed?
    
    char linux_str[] = "Linux by write\n";
    
    // write
    // 1 = STDOUT_FILENO - изначально открытый файловый дескриптор соответствующий stdout
    // linux_str - указатель на начало данных, 
    // sizeof(linux_str) - размер данных, которые хотим записать
    // ВАЖНО, что write может записать не все данные 
    //        и тогда его надо перезапустить
    //        но в данном примере этого нет
    // Подробнее в `man 2 write`
    write(1, linux_str, sizeof(linux_str) - 1); 
    // exit(0); // 1. Что выведется если раскомментировать?
    // _exit(0); // 2. Что выведется если раскомментировать это?
        
    if (argc < 2) {
        fprintf(stderr, "Need at least 2 arguments\n");
        return 1;
    }
    int fd = open(argv[1], O_RDONLY); // открываем файл и получаем связанный файловый дескриптор
                                      // O_RDONLY - флаг о том, что открываем в read-only режиме
                                      // подробнее в `man 2 open`
    if (fd < 0) {
        perror("Can't open file"); // Выводит указанную строку в stderr 
                                   // + добавляет сообщение и последней произошедшей ошибке 
                                   // ошибка хранится в errno
        return -1;
    }
    
    char buffer[4096];
    int bytes_read = read(fd, buffer, sizeof(buffer)); // fd - файловый дескриптор выше открытого файла
                                                       // 2 и 3 аргументы как во write
                                                       // Так же как и write может прочитать МЕНЬШЕ
                                                       //   чем запрошено в 3м аргументе
                                                       //   это может быть связано как с концом файла
                                                       //   так и с каким-то более приоритетным событием
    if (bytes_read < 0) {
        perror("Error reading file");
        close(fd); // закрываем файл связанный с файловым дескриптором. Ну или не файл. 
                   // Стандартные дескрипторы 0, 1, 2 тоже можно так закрывать
        return -1;
    }
    char buffer2[4096];
    // формирование строки с текстом
    int written_bytes = snprintf(buffer2, sizeof(buffer2), 
        "%d bytes read: '''%.*s'''\n", bytes_read, bytes_read, buffer);
    write(1, buffer2, written_bytes);
    close(fd);
    return 0;
}

Run: `gcc linux_example.c -o linux_example.exe`

Run: `echo -n "Hello students!" > linux_example_input_001.txt`

Run: `./linux_example.exe linux_example_input_001.txt`

Linux by write
15 bytes read: '''Hello students!'''
Linux by printf

### Экзотический пример-игрушка

In [95]:
%%cpp strange_example.c
%run gcc strange_example.c -o strange_example.exe
%run echo "Hello world!" > a.txt
%run ./strange_example.exe 5< a.txt > strange_example.out
%run cat strange_example.out

#include <unistd.h>
#include <stdio.h>

int main(int argc, char *argv[])
{ 
    char buffer[4096];
    int bytes_read = read(5, buffer, sizeof(buffer)); // not standart opened fd
    if (bytes_read < 0) {
        perror("Error reading file");
        return -1;
    }
    int written_bytes = write(1, buffer, bytes_read);
    if (written_bytes < 0) {
        perror("Error writing file");
        return -1;
    }
    return 0;
}

Run: `gcc strange_example.c -o strange_example.exe`

Run: `echo "Hello world!" > a.txt`

Run: `./strange_example.exe 5< a.txt > strange_example.out`

Run: `cat strange_example.out`

Hello world!


## <a name="attrs"></a> Атрибуты файла

При открытии файла с флагом создания (O_WRONLY | O_CREAT) важно адекватно проставлять маску прав доступа. Давайте с ней разберемся.

Заметка о правописании: **Attribute, но атрибут**

In [96]:
!echo "Hello jupyter!" > a.txt  # создаем файлик с обычными "настройками"
!mkdir b_dir 2> /dev/null

import os  # В модуле os есть почти в чистом виде почти все системные вызовы: write, read, open...
from IPython.display import display

%p os.stat("a.txt") # Атрибуты файла `a.txt`
%p oct(os.stat("a.txt").st_mode)  # Интересны последние три восьмеричные цифры. 664 - это обычные атрибуты прав

%p oct(os.stat("./linux_example.exe").st_mode)  # Аттрибуты прав исполняемого файла

%p oct(os.stat("b_dir").st_mode)  # Забавный факт, но все могут "исполнять директорию". [Более подробно на stackoverflow](https://unix.stackexchange.com/questions/21251/execute-vs-read-bit-how-do-directory-permissions-in-linux-work)


`os.stat("a.txt") = os.stat_result(st_mode=33204, st_ino=4719095, st_dev=2053, st_nlink=1, st_uid=1000, st_gid=1000, st_size=15, st_atime=1604590800, st_mtime=1604590801, st_ctime=1604590801)`  # Атрибуты файла `a.txt`

`oct(os.stat("a.txt").st_mode) = 0o100664`  # Интересны последние три восьмеричные цифры. 664 - это обычные атрибуты прав

`oct(os.stat("./linux_example.exe").st_mode) = 0o100775`  # Аттрибуты прав исполняемого файла

`oct(os.stat("b_dir").st_mode) = 0o40775`  # Забавный факт, но все могут "исполнять директорию". [Более подробно на stackoverflow](https://unix.stackexchange.com/questions/21251/execute-vs-read-bit-how-do-directory-permissions-in-linux-work)

In [97]:
!stat ./linux_example.exe


  File: ./linux_example.exe
  Size: 17064     	Blocks: 40         IO Block: 4096   regular file
Device: 805h/2053d	Inode: 4719091     Links: 1
Access: (0775/-rwxrwxr-x)  Uid: ( 1000/pechatnov)   Gid: ( 1000/pechatnov)
Access: 2020-11-05 18:40:00.008651022 +0300
Modify: 2020-11-05 18:39:59.664645536 +0300
Change: 2020-11-05 18:39:59.664645536 +0300
 Birth: -


In [98]:
%%cpp linux_file_hello_world.c
%run gcc linux_file_hello_world.c -o linux_file_hello_world.exe
%run ./linux_file_hello_world.exe
%run cat linux_file_hello_world.out

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>

int main(int argc, char *argv[])
{   
    int fd = open("linux_file_hello_world.out", O_WRONLY | O_CREAT, S_IRUSR | S_IWUSR | S_IRGRP | S_IWGRP | S_IROTH); 
    // S_IRUSR | S_IWUSR | S_IRGRP | S_IWGRP | S_IROTH == 0664
    // попробуйте не указывать 0664   
    // (ошибка такая же как в printf("%d");)
    // для справки `man 2 open`
     
    if (fd < 0) {
        perror("Can't open file");
        return -1;
    }
    char buffer[] = "Hello world!";
    int bytes_written = write(fd, buffer, sizeof(buffer));
    if (bytes_written < 0) {
        perror("Error writing file");
        close(fd);
        return -1;
    }
    printf("Bytes written: %d (expected %d)\n", bytes_written, (int)sizeof(buffer));
    close(fd);
    return 0;
}

Run: `gcc linux_file_hello_world.c -o linux_file_hello_world.exe`

Run: `./linux_file_hello_world.exe`

Bytes written: 13 (expected 13)


Run: `cat linux_file_hello_world.out`

Hello world! 

In [99]:
oct(os.stat("linux_file_hello_world.out").st_mode)

'0o100664'

## <a name="lseek"></a> lseek - чтение с произвольной позиции в файле

Смотрит на второй символ в файле, читает его, интерпретирует как цифру и увеличивает эту цифру на 1.

In [100]:
%%cpp lseek_example.c
%run gcc lseek_example.c -o lseek_example.exe
%run ./lseek_example.exe b.txt
%run cat b.txt

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>

int main(int argc, char *argv[])
{   
    assert(argc >= 2);
    // O_RDWR - открытие файла на чтение и запись одновременно
    int fd = open(argv[1], O_RDWR | O_CREAT, S_IRUSR | S_IWUSR | S_IRGRP | S_IWGRP | S_IROTH); 
    
    // Перемещаемся на конец файла, получаем позицию конца файла - это размер файла
    int size = lseek(fd, 0, SEEK_END);
    
    printf("File size: %d\n", size);
    
    // если размер меньше 2, то дописываем цифры
    if (size < 2) {
        const char s[] = "10";
        lseek(fd, 0, SEEK_SET);
        write(fd, s, sizeof(s) - 1);
        printf("Written bytes: %d\n", (int)sizeof(s) - 1);    
        size = lseek(fd, 0, SEEK_END);
        printf("File size: %d\n", size);
    }
    
    // читаем символ со 2й позиции
    lseek(fd, 1, SEEK_SET);
    char c;
    read(fd, &c, 1);
    c = (c < '0' || c > '9') ? '0' : ((c - '0') + 1) % 10 + '0';
    
    // записываем символ в 2ю позицию
    lseek(fd, 1, SEEK_SET);
    write(fd, &c, 1);
    
    close(fd);
    return 0;
}

Run: `gcc lseek_example.c -o lseek_example.exe`

Run: `./lseek_example.exe b.txt`

File size: 6


Run: `cat b.txt`

h6llo


In [101]:
!echo "hello" > b.txt

## <a name="lsof"></a> Список открытых файлов

In [139]:
%%cpp simple_open.c
%run gcc simple_open.c -o simple_open.exe
%run ./simple_open.exe
%run ./simple_open.exe < a.txt
%run ./simple_open.exe < a.txt 2> b.txt

#include <sys/types.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>


void apply_lsof() {
    char cmd[100];
    snprintf(cmd, sizeof(cmd), "lsof -p %d", getpid());
    system(cmd);
}


int main()
{  
    apply_lsof();
    return 0;
}

Run: `gcc simple_open.c -o simple_open.exe`

Run: `./simple_open.exe`

COMMAND     PID      USER   FD   TYPE DEVICE SIZE/OFF    NODE NAME
simple_op 62730 pechatnov  cwd    DIR    8,5     4096 4718759 /home/pechatnov/vbox/caos/sem10-low-level-io
simple_op 62730 pechatnov  rtd    DIR    8,5     4096       2 /
simple_op 62730 pechatnov  txt    REG    8,5    16880 4723899 /home/pechatnov/vbox/caos/sem10-low-level-io/simple_open.exe
simple_op 62730 pechatnov  mem    REG    8,5  2029224 1840863 /usr/lib/x86_64-linux-gnu/libc-2.31.so
simple_op 62730 pechatnov  mem    REG    8,5   191472 1840650 /usr/lib/x86_64-linux-gnu/ld-2.31.so
simple_op 62730 pechatnov    0u   CHR  136,2      0t0       5 /dev/pts/2
simple_op 62730 pechatnov    1u   CHR  136,2      0t0       5 /dev/pts/2
simple_op 62730 pechatnov    2u   CHR  136,2      0t0       5 /dev/pts/2


Run: `./simple_open.exe < a.txt`

COMMAND     PID      USER   FD   TYPE DEVICE SIZE/OFF    NODE NAME
simple_op 62735 pechatnov  cwd    DIR    8,5     4096 4718759 /home/pechatnov/vbox/caos/sem10-low-level-io
simple_op 62735 pechatnov  rtd    DIR    8,5     4096       2 /
simple_op 62735 pechatnov  txt    REG    8,5    16880 4723899 /home/pechatnov/vbox/caos/sem10-low-level-io/simple_open.exe
simple_op 62735 pechatnov  mem    REG    8,5  2029224 1840863 /usr/lib/x86_64-linux-gnu/libc-2.31.so
simple_op 62735 pechatnov  mem    REG    8,5   191472 1840650 /usr/lib/x86_64-linux-gnu/ld-2.31.so
simple_op 62735 pechatnov    0r   REG    8,5       15 4719095 /home/pechatnov/vbox/caos/sem10-low-level-io/a.txt
simple_op 62735 pechatnov    1u   CHR  136,2      0t0       5 /dev/pts/2
simple_op 62735 pechatnov    2u   CHR  136,2      0t0       5 /dev/pts/2


Run: `./simple_open.exe < a.txt 2> b.txt`

COMMAND     PID      USER   FD   TYPE DEVICE SIZE/OFF    NODE NAME
simple_op 62740 pechatnov  cwd    DIR    8,5     4096 4718759 /home/pechatnov/vbox/caos/sem10-low-level-io
simple_op 62740 pechatnov  rtd    DIR    8,5     4096       2 /
simple_op 62740 pechatnov  txt    REG    8,5    16880 4723899 /home/pechatnov/vbox/caos/sem10-low-level-io/simple_open.exe
simple_op 62740 pechatnov  mem    REG    8,5  2029224 1840863 /usr/lib/x86_64-linux-gnu/libc-2.31.so
simple_op 62740 pechatnov  mem    REG    8,5   191472 1840650 /usr/lib/x86_64-linux-gnu/ld-2.31.so
simple_op 62740 pechatnov    0r   REG    8,5       15 4719095 /home/pechatnov/vbox/caos/sem10-low-level-io/a.txt
simple_op 62740 pechatnov    1u   CHR  136,2      0t0       5 /dev/pts/2
simple_op 62740 pechatnov    2w   REG    8,5        0 4723860 /home/pechatnov/vbox/caos/sem10-low-level-io/b.txt


# <a name="win"></a> Windows

* Вместо файловых дескрипторов - HANDLE (вроде это просто void*)
* Много алиасов для типов вроде HANDLE, DWORD, BOOL, LPTSTR, LPWSTR
* Очень много аргументов у всех функций
* Плохая документация, гуглится все плохо
* Надо установить `wine` и `mingw-w64`
  * `sudo apt install gcc-mingw-w64`
  * `sudo apt install wine64`

In [102]:
%%cpp winapi_example.c
%run i686-w64-mingw32-gcc winapi_example.c -o winapi_example.exe
%run echo "Hello students!" > winapi_example_input_001.txt
%run WINEDEBUG=-all wine winapi_example.exe winapi_example_input_001.txt

#include <windows.h>
#include <stdio.h>

int main(int argc, char *argv[])
{
#ifdef WIN32
    printf("Defined WIN32\n");
#else
    printf("Not WIN32\n");
#endif
    if (argc < 2) {
        printf("Need at least 2 arguments\n");
        return 1;
    }
    HANDLE fileHandle = CreateFileA(
        argv[1], GENERIC_READ, FILE_SHARE_READ, NULL,
        OPEN_EXISTING, FILE_ATTRIBUTE_NORMAL, NULL);
    if (fileHandle == INVALID_HANDLE_VALUE) {
        char errorBuffer[1024];
        if (!FormatMessage(FORMAT_MESSAGE_FROM_SYSTEM | FORMAT_MESSAGE_IGNORE_INSERTS,
                           NULL, GetLastError(),
                           MAKELANGID(LANG_NEUTRAL, SUBLANG_DEFAULT),
                           errorBuffer, sizeof(errorBuffer), NULL))
        {
            printf("Format message failed with 0x%x\n", GetLastError());
            return -1;
        }
        printf("Can't open file: %s\n", errorBuffer);
        return -1;
    }
    
    char buffer[4096];
    DWORD bytes_read;
    BOOL success;
    success = ReadFile(fileHandle, buffer, sizeof(buffer),
                       &bytes_read, NULL);
    if (!success) {
        perror("Error reading file"); // Это ошибка, perror смотрит в errno, а не в GetLastError()
        CloseHandle(fileHandle);
        return -1;
    }
    printf("Bytes read: %d\n'''%.*s'''\n", bytes_read, bytes_read, buffer);
    CloseHandle(fileHandle);
    return 0;
}

Run: `i686-w64-mingw32-gcc winapi_example.c -o winapi_example.exe`

Run: `echo "Hello students!" > winapi_example_input_001.txt`

Run: `WINEDEBUG=-all wine winapi_example.exe winapi_example_input_001.txt`

=Defined WIN32
Bytes read: 16
'''Hello students!
'''
>

## <a name="hw"></a> Комментарии к ДЗ
* 0777 - реджект, исполняемый файл создавать не надо
* open -> close. Нужно закрывать все файлы
* Рассматривать ошибки и случаи неполного чтения в случае работы НЕ с обычными файлами с файловой системы. (Например, если это может быть ввод с терминала или пайпа).
* files-io/read-filter-write - запрещено читать по одному символу. И читать все срарзу в память тоже не надо. Делайте буфеизацию.
* files-io/sort-file-contents - запрещено читать и выводить по одному инту - ужасно неэффективно.